In [1]:
import sys, os
sys.path.append(os.path.relpath("../Repo/src/PDRO_Newsvendor/"))
from ambiguity_set import *
from demand_RV import *
from DRMPNVP import *
from MPNVP import *

import datetime

num_processors = 32
gurobi_cores = 4
loop_cores = int(num_processors / gurobi_cores)
timeout = 4 * 60 * 60

T_vals = range(2, 5)
mu_0_range = range(1, 21)
sig_0_range = range(1, 11)
num_omega0 = 3

PWL_gap_vals = list(reversed([0.1, 0.25, 0.5]))
disc_pts_vals = [3, 5, 10]
p_range = list(100 * np.array(range(1, 3)))
h_range = list(100 * np.array(range(1, 3)))
b_range = list(100 * np.array(range(1, 3)))
W_range = [4000]
N_vals = [10, 25, 50]

omega0_all = [mu_sig_combos(mu_0_range, sig_0_range, T_) 
              for T_ in T_vals]

omega0_vals = []
for T_ in T_vals:
    np.random.seed(T_vals.index(T_))
    indices = np.random.choice(
        range(len(omega0_all[T_vals.index(T_)])), num_omega0)
    omega0_vals.append(
        [omega0_all[T_vals.index(T_)][i] for i in indices]
    )

inputs = [(mu_0, sig_0, T_, W, w, p, h, b, PWL_gap, n_pts,
           0.001, timeout, gurobi_cores, N, 0.05)
          for T_ in T_vals
          for (mu_0, sig_0) in omega0_vals[T_vals.index(T_)]
          for p in p_range for h in h_range 
          for b in b_range for PWL_gap in PWL_gap_vals
          for N in N_vals for n_pts in disc_pts_vals
          for w in [list(100*np.array(range(1, T_+1))[::-1])]
          for W in W_range if b > max([w[t] - w[t+1] for t in range(T_-1)])]

for i in range(len(inputs)):
    inputs[i] = tuple([i] + list(inputs[i]))
num_inputs = len(inputs)

names = ["ind", "rep", "num_reps", "mu_0", "sig_0", "T", "W", "w", "p", "h", "b", 
         "PWL_gap", "n_pts", "CS_tol", "solver_timeout", "N",
         "alpha", "num_dists_full", "num_dists_reduced", "PWL_q", "PWL_worst", "PWL_obj", 
         "PWL_true_worst", "PWL_true_worst_obj", "PWL_tt", "PWL_TO", "PWL_true_obj",
        "CS_q", "CS_worst", "CS_obj", "CS_true_worst", "CS_true_worst_obj",
         "CS_tt", "CS_TO", "CS_reason", "CS_true_obj", "fd_q", "fd_worst", "fd_obj", 
         "fd_true_worst", "fd_true_worst_obj", "fd_tt", "fd_true_obj"
]  

num_instances = 10
num_reps = 1

if num_reps > 1:
    results_file = "results_reps.txt"
    count_file = "count_reps.txt"
    repeated_inputs = []
    counter = 0
    for T_ in T_vals:
        inps = [i for i in inputs if i[3] == T_][:num_instances]
        for rep in range(num_reps):
            repeated_inputs += [tuple([i[0], rep, num_reps] + list(i)[1:]) for i in inps]
else:
    results_file = "test_res.txt"
    count_file = "test_count.txt"
    inps = inputs
    repeated_inputs = [tuple([i[0], 0, 1] + list(i)[1:]) for i in inps]

inputs = repeated_inputs

test_full = [i for i in inputs if (i[names.index("T")], i[names.index("n_pts")]) != (4, 10)]

In [2]:
inp = test_full[3]

(index, rep, num_reps, mu_0, sig_0, T, W, w, p, h, b, 
     PWL_gap, n_pts, CS_tol, timeout, solver_cores, N, 
     alpha) = inp

# Normal Instance

In [10]:
dist = "normal"
omega_0 = (mu_0, sig_0)
#construct MLEs and confidence set
X = demand_RV(dist, T, (mu_0, sig_0), N, seed=index)
X.compute_mles()
AS = ambiguity_set(X, "confidence_set", alpha, n_pts, timeout)
AS.construct_base_set()
AS.construct_confidence_set()
AS.compute_extreme_distributions()
AS.reduce()
print("built and reduced ambiguity set.\n")

num_dists = len(AS.reduced)
num_dists

built and reduced ambiguity set.



21

In [4]:
DRO_problem = DRMPNVP(T, W, w, p, h, b, PWL_gap, timeout-AS.time_taken, 
                          solver_cores, dist, AS)
    
PWL_sol=DRO_problem.PWL_solve(AS.reduced)

In [5]:
PWL_sol

[array([ 0.93146158, 13.60877346]),
 2869.3137787254786,
 ((2.2339458578054767, 12.306289178930502),
  (6.315605209351114, 10.218856688320994)),
 0.146,
 False]

In [6]:
CS_sol=DRO_problem.CS_solve(AS.reduced[0])
CS_sol

[array([ 0.93146158, 13.60877346]),
 ((2.2339458578054767, 12.306289178930502),
  (6.315605209351114, 10.218856688320994)),
 2847.1521636489324,
 0.141,
 False,
 'repeat']

In [3]:
fd_problem = MPNVP(T, W, w, p, h, b, PWL_gap, timeout, 
                       solver_cores, dist, X.mle, budget_tol=1e-6, 
                       tau=0.5, alpha_min=1e-100, alpha_init=0.1)
s=time.perf_counter()
fd_q = fd_problem.alfares_solve()
e=time.perf_counter()

In [4]:
fd_q

array([ 1.2761263 , 13.26410874])

# Poisson Instance

In [3]:
dist = "poisson"
lam_0 = tuple([(T - t) * 10 for t in range(T)]) 
#construct MLEs and confidence set
X = demand_RV(dist, T, lam_0, N, seed=index * num_reps + rep)
X.compute_mles()
AS = ambiguity_set(X, "confidence_set", alpha, n_pts, timeout=timeout)
AS.construct_base_set()
AS.construct_confidence_set()
AS.reduce()
AS.compute_extreme_distributions()

num_dists = len(AS.reduced)

DRO_problem = DRMPNVP(T, W, w, p, h, b, 1, timeout, 
                      gurobi_cores, dist, AS)

In [4]:
PWL_sol=DRO_problem.PWL_solve(AS.reduced)
PWL_sol

[array([14., 12.]), 3560.5890076550377, (19.68, 9.32), 0.032, False]

In [5]:
CS_sol=DRO_problem.CS_solve(AS.reduced[0])
CS_sol

[array([14., 12.]), (19.68, 9.32), 3560.5890076550377, 0.027, False, 'optimal']

In [6]:
fd_problem = MPNVP(T, W, w, p, h, b, PWL_gap, timeout, 
                       solver_cores, dist, X.mle, budget_tol=1e-6, 
                       tau=0.5, alpha_min=1e-100, alpha_init=0.1)
s=time.perf_counter()
fd_q = fd_problem.alfares_solve()
e=time.perf_counter()
fd_q

array([14., 12.])